In [4]:
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
df=pd.read_csv('../data/batch2_contracts_125rows.csv')
df.head()

,contract_id,raw_text,apr,term_months,monthly_payment,penalty_clause,recommended_action,risk_flag
0,1,This contract between Prime Auto and Emily Sto...,10.49,24,959,NaN,Approve,Low
1,2,This contract between ABC Motors and Priya Sha...,3.78,24,804,Early termination fee $300,Approve,Low
2,3,This contract between DriveEasy Finance and Sa...,5.41,24,774,Late fee $50,Reject,High
3,4,This contract between National Motors and Alex...,5.26,48,1028,Late fee $25,Approve,High
4,5,This contract between National Motors and John...,5.97,36,1180,NaN,Approve,Low


In [6]:
df.info()
df.describe()
df.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125 entries, 0 to 124
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   contract_id         125 non-null    int64  
 1   raw_text            125 non-null    object 
 2   apr                 125 non-null    float64
 3   term_months         125 non-null    int64  
 4   monthly_payment     125 non-null    int64  
 5   penalty_clause      93 non-null     object 
 6   recommended_action  125 non-null    object 
 7   risk_flag           125 non-null    object 
dtypes: float64(1), int64(3), object(4)
memory usage: 7.9+ KB


,contract_id,raw_text,apr,term_months,monthly_payment,penalty_clause,recommended_action,risk_flag
27,28,This contract between ABC Motors and Emily Sto...,5.54,60,1026,Early termination fee $300,Approve,Low
43,44,This contract between DriveEasy Finance and Pr...,3.94,24,960,Late fee $50,Review manually,Medium
110,111,This contract between ABC Motors and Emily Sto...,2.91,48,923,NaN,Reject,Medium
45,46,This contract between CarHub Leasing and David...,3.13,60,1049,NaN,Approve,Low
100,101,This contract between CarHub Leasing and Priya...,5.20,60,483,NaN,Approve,Low


In [ ]:
def parse_contract(text):
    result = {}

    # Extract APR
    import re
    apr_match = re.search(r'APR (\d+\.?\d*)%', text)
    result['apr_extracted'] = float(apr_match.group(1)) if apr_match else None

    # Extract term
    term_match = re.search(r'(\d+) months', text)
    result['term_extracted'] = int(term_match.group(1)) if term_match else None

    # Extract payment
    payment_match = re.search(r'monthly payment \$?(\d+)', text)
    result['monthly_payment_extracted'] = int(payment_match.group(1)) if payment_match else None

    # Extract penalty
    penalty_match = re.search(r'(Late fee \$\d+|Early termination fee \$\d+|None)', text)
    result['penalty_extracted'] = penalty_match.group(1) if penalty_match else None

    return resultD

parsed = df['raw_text'].apply(parse_contract)
parsed_df = pd.DataFrame(parsed.tolist())
parsed_df.head()

,apr_extracted,term_extracted,monthly_payment_extracted,penalty_extracted
0,10.49,24,959,None
1,3.78,24,804,Early termination fee $300
2,5.41,24,774,Late fee $50
3,5.26,48,1028,Late fee $25
4,5.97,36,1180,None


In [4]:
def risk_rule(row):
    if row['apr'] > 10:
        return "HIGH"
    if row['monthly_payment'] > 900:
        return "MEDIUM"
    return "LOW"

df['rule_based_risk'] = df.apply(risk_rule, axis=1)
df[['raw_text', 'rule_based_risk']].head()

,raw_text,rule_based_risk
0,This contract between Prime Auto and Emily Sto...,HIGH
1,This contract between ABC Motors and Priya Sha...,LOW
2,This contract between DriveEasy Finance and Sa...,LOW
3,This contract between National Motors and Alex...,MEDIUM
4,This contract between National Motors and John...,MEDIUM


In [6]:
final_output = df[['contract_id',
                   'raw_text',
                   'apr',
                   'term_months',
                   'monthly_payment',
                   'penalty_clause',
                   'recommended_action',
                   'risk_flag']]


In [10]:
final_output.to_csv("Milestone1_pavan-srinivas.csv", index=False)